In [113]:
import pandas as pd, numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from stock_keys import api_key
from stock_saver import Stock
import stock_chart as sc
import stock_indicators
import math

ticker = 'NSE:INFY'
intraday = False
day, month, year = 6, 6, 2019
st = Stock(ticker, api_key, intraday, None)
df = pd.DataFrame(data=st.df)
df = df[df.index.map(lambda x: (df.date[x].year) == year and (df.date[x].month) == month and (df.date[x].day) == day)]
print('Filtered values:', len(df.index))
def compute_rate(values):
    return((values[1] - values[0]) / values[0])
output_notebook()

[NSE:INFY] Intraday : 2019-06-06 05:59:00
[NSE:INFY] 2999 values loaded.
Filtered values: 360


Loading BokehJS ...

In [114]:
def compute_rate(values):
    return((values[1] - values[0]) / values[0])

df['green'] = (df.close >= df.open)
df['red'] = (df.open > df.close)
df['open_rate'] = df.open.rolling(window=(2)).apply(compute_rate, raw=True).fillna(0)
df['high_rate'] = df.high.rolling(window=(2)).apply(compute_rate, raw=True).fillna(0)
df['low_rate'] = df.low.rolling(window=(2)).apply(compute_rate, raw=True).fillna(0)
df['close_rate'] = df.close.rolling(window=(2)).apply(compute_rate, raw=True).fillna(0)
df['volume_rate'] = df.volume.rolling(window=(2)).apply(compute_rate, raw=True).fillna(0)

def compute_correctness(df):
    # We observed across stocks that only just a bit more than half the trades predicts correct direction.
    # A bit less than half of the trades lose money for other half to gain.
    correctness = []
    for i, row in df.iterrows():
        avg = (row.open + row.close) / 2
        up = row.high - avg
        down = avg - row.low
        total = up + down
        if total == 0:
            correctness.append(0)
            continue
        if up > down:
            if row.close >= row.open:
                correctness.append(up / total)
            else:
                correctness.append(down / total)
        else:
            if row.close < row.open:
                correctness.append(down / total)
            else:
                correctness.append(up / total)
    df['correctness'] = correctness

compute_correctness(df)
# Insteaad compute correct %

In [117]:
# We found that just
print(df.correctness.mean())
df

0.5260100952466178


,date,open,high,low,close,volume,green,red,open_rate,high_rate,low_rate,close_rate,volume_rate,correctness
2910,2019-06-06 00:00:00.000000,736.05,736.35,735.75,736.00,21166,False,True,0.000000,0.000000,0.000000,0.000000,0.000000,0.458333
2911,2019-06-06 00:01:00.000000,736.00,736.00,735.60,735.65,14521,False,True,-0.000068,-0.000475,-0.000204,-0.000476,-0.313947,0.562500
2912,2019-06-06 00:02:00.000000,735.70,735.75,735.45,735.55,29999,False,True,-0.000408,-0.000340,-0.000204,-0.000136,1.065905,0.583333
2913,2019-06-06 00:03:00.000000,735.50,735.55,735.30,735.50,14903,True,False,-0.000272,-0.000272,-0.000204,-0.000068,-0.503217,0.200000
2914,2019-06-06 00:04:00.000000,735.50,735.60,735.40,735.45,28243,False,True,0.000000,0.000068,0.000136,-0.000068,0.895122,0.375000
2915,2019-06-06 00:05:00.000000,735.45,735.50,734.80,735.10,34547,False,True,-0.000068,-0.000136,-0.000816,-0.000476,0.223206,0.678571
2917,2019-06-06 00:06:00.000001,735.10,735.20,735.00,735.00,17462,False,True,-0.000476,-0.000408,0.000272,-0.000136,-0.494544,0.250000
2918,2019-06-06 00:07:00.000000,735.00,735.10,734.35,734.65,83712,False,True,-0.000136,-0.000136,-0.000884,-0.000476,3.793953,0.633333
2919,2019-06-06 00:08:00.000000,734.60,734.80,734.60,734.80,15394,True,False,-0.000544,-0.000408,0.000340,0.000204,-0.816108,0.500000
2920,2019-06-06 00:09:00.000000,734.80,735.20,734.75,735.20,20866,True,False,0.000272,0.000544,0.000204,0.000544,0.355463,0.444444


In [118]:
p = figure(x_axis_type="datetime", plot_width=1000, plot_height=350, x_range = (df.date.min(), df.date.max()))
p.line('date', 'correctness', source = ColumnDataSource(df), line_color="green")
show(p)

p = figure(x_axis_type="datetime", plot_width=1000, plot_height=350, x_range = (df.date.min(), df.date.max()), y_range = (df.low.min(), df.high.max()))
p = sc.set_title(p, st.ticker)
# trends lies between local maximas and minimas of the 10min correlation
p = sc.draw_price(p, df)
p = sc.draw_volume(p, df, False)
# p.circle(blues['date'], blues['price'], size=15, color="orange", alpha=0.8)
show(p)